In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import (
    roc_curve,
    auc,
    precision_recall_curve,
    average_precision_score,
    brier_score_loss,
)
from sklearn.calibration import calibration_curve
import numpy as np
import pandas as pd
import os
import sys

import equiboots as eqb

### Fetching Dataset

In [ ]:
adult_x = pd.read_parquet("../data/processed/X.parquet")
adult_y = pd.read_parquet("../data/processed/y_income.parquet")

In [ ]:
from adult_income.functions import find_best_model, mlflow_load_model

In [ ]:
def return_best_model(outcome, metric, mlruns_location):

    outcome = "income"
    experiment_name = outcome + "_model"

    run_name, estimator_name = find_best_model(
        experiment_name, metric, mlruns_location=mlruns_location
    )

    model_name = f"{estimator_name}_{outcome}"
    best_model = mlflow_load_model(
        experiment_name, run_name, model_name, mlruns_location=mlruns_location
    )
    return best_model

In [ ]:
best_model = return_best_model("income", "valid Average Precision", "../mlruns/models/")


test_x, test_y = best_model.get_test_data(adult_x, adult_y)

In [ ]:
y_pred = best_model.predict(test_x)
y_prob = best_model.predict_proba(test_x)[:, 1]
y_true = test_y.to_numpy().ravel()
fairness_df = test_x[["race", "sex"]]

In [ ]:
fairness_df = fairness_df.reset_index()

In [ ]:
eq = eqb.EquiBoots(
    y_true=y_true,
    y_prob=y_prob,
    y_pred=y_pred,
    fairness_df=fairness_df,
    fairness_vars=["race", "sex"],
)
eq.grouper(groupings_vars=["race", "sex"])
sliced_data = eq.slicer("race")

In [ ]:
y_true

In [ ]:
eqb.eq_plot_roc_auc(
    data=sliced_data,
    title="ROC Curve by Race",
    decimal_places=3,
)

In [ ]:
# Precision-Recall plot
fig2 = eqb.eq_plot_precision_recall(
    data=sliced_data,
    title="Precision-Recall Curve by Race",
    decimal_places=3,
    dpi=100,
)

In [ ]:
# Calibration plot
fig3 = eqb.eq_calibration_curve_plot(
    data=sliced_data,
    n_bins=10,
    title="Calibration Curve by Race",
    tick_fontsize=8,
    decimal_places=3,
)
fig3

### Bootstrapped Graphs - Stratified

In [ ]:
eq2 = eqb.EquiBoots(
    y_true,
    y_prob,
    y_pred,
    fairness_df,
    fairness_vars=["race", "sex"],
    reference_groups=["White", "Male"],
    task="binary_classification",
    bootstrap_flag=True,
    num_bootstraps=10,
    boot_sample_size=2000,
    balanced=False,
)

# Set seeds
int_list = np.linspace(200, 300, num=10, dtype=int).tolist()


eq2.set_fix_seeds(int_list)

print("seeds", eq2.seeds)

eq2.grouper(groupings_vars=["race", "sex"])
sliced_data = eq.slicer("race")

data = eq2.slicer("race")
race_metrics = eq2.get_metrics(data)

dispa = eq2.calculate_disparities(race_metrics, "race")

melted = pd.DataFrame(dispa).melt()
df = melted["value"].apply(pd.Series).assign(attribute_value=melted["variable"])

In [ ]:
eqb.eq_disparity_metrics_plot(
    dispa,
    metric_cols=[
        "Accuracy_ratio",
        "Precision_ratio",
        "Predicted Prevalence_ratio",
        "Prevalence_ratio",
    ],
    name="race",
    categories="all",
    figsize=(10, 9),
    strict_layout=False,
    plot_kind="violinplot",
)

In [ ]:
sliced = eq2.slicer(slicing_var="race")
eqb.eq_plot_bootstrapped_roc_curves(sliced, bar_every=100)

In [ ]:
eqb.eq_plot_bootstrapped_pr_curves(sliced, bar_every=100)

In [ ]:
eqb.eq_plot_bootstrapped_calibration_curves(sliced, n_bins=5)

### Bootstrapped Graphs - Balanced

In [ ]:
eq2 = eqb.EquiBoots(
    y_true,
    y_prob,
    y_pred,
    fairness_df,
    fairness_vars=["race", "sex"],
    reference_groups=["White", "Male"],
    task="binary_classification",
    bootstrap_flag=True,
    num_bootstraps=10,
    boot_sample_size=2000,
    balanced=True,
)

# Set seeds
int_list = np.linspace(200, 300, num=10, dtype=int).tolist()


eq2.set_fix_seeds(int_list)

print("seeds", eq2.seeds)

eq2.grouper(groupings_vars=["race", "sex"])
sliced_data = eq.slicer("race")

data = eq2.slicer("race")
race_metrics = eq2.get_metrics(data)

dispa = eq2.calculate_disparities(race_metrics, "race")

melted = pd.DataFrame(dispa).melt()
df = melted["value"].apply(pd.Series).assign(attribute_value=melted["variable"])

In [ ]:
eqb.eq_disparity_metrics_plot(
    dispa,
    metric_cols=[
        "Accuracy_ratio",
        "Precision_ratio",
        "Predicted Prevalence_ratio",
        "Prevalence_ratio",
    ],
    name="race",
    categories="all",
    # figsize=(20, 5),
    # strict_layout=False,
    plot_kind="violinplot",
)

In [ ]:
sliced = eq2.slicer(slicing_var="race")
eqb.eq_plot_bootstrapped_roc_curves(sliced, bar_every=100)

In [ ]:
eqb.eq_plot_bootstrapped_pr_curves(sliced, bar_every=100)

In [ ]:
eqb.eq_plot_bootstrapped_calibration_curves(sliced, n_bins=5)